In [2]:
import numpy as np
from scipy.io import loadmat
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Load SVHN dataset from local .mat files
def load_svhn_data(train_path, test_path):
    train_data = loadmat(train_path)
    test_data = loadmat(test_path)

    x_train = train_data['X']  # shape (32, 32, 3, num_samples)
    y_train = train_data['y']  # shape (num_samples, 1)

    x_test = test_data['X']  # shape (32, 32, 3, num_samples)
    y_test = test_data['y']  # shape (num_samples, 1)

    # Reshape and transpose the data
    x_train = np.transpose(x_train, (3, 0, 1, 2))
    x_test = np.transpose(x_test, (3, 0, 1, 2))

    # Correct labels: SVHN labels are 1-10; convert 10 to 0
    y_train = np.where(y_train == 10, 0, y_train).flatten()
    y_test = np.where(y_test == 10, 0, y_test).flatten()

    return x_train, y_train, x_test, y_test

# Specify the paths to your downloaded SVHN .mat files
train_path = 'path/to/train_32x32.mat'
test_path = 'path/to/test_32x32.mat'

# Load the data
x_train, y_train, x_test, y_test = load_svhn_data(train_path, test_path)

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Load VGG16 model pre-trained on ImageNet, without the top layers
base_model = VGG16(weights='imagenet', include_top=False)

# Build the model
model = Sequential()
model.add(InputLayer(input_shape=(32, 32, 3)))
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation to reduce overfitting
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Fit the model using data augmentation
batch_size = 64
epochs = 3
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/train_32x32.mat'